In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, cross_val_score
from nltk.corpus import stopwords

corpus = pd.read_csv('corpus.csv.gz', compression='gzip')
corpus = corpus[corpus['qual_a_melhor_classificao_para_esse_texto:confidence'] == 1]
stopwords = stopwords.words("portuguese")

In [2]:
# fix labels to binary
def classFit(x):
    if x['qual_a_melhor_classificao_para_esse_texto'] == "diario":
        return 1
    else:
        return -1
    
corpus['class'] = corpus.apply(classFit,axis=1)
target = corpus['class'].values

print(corpus['qual_a_melhor_classificao_para_esse_texto'].values[:2])
print(corpus['class'][:2])

['diario' 'diario']
0    1
4    1
Name: class, dtype: int64


In [3]:
data = TfidfVectorizer(max_features=1000, strip_accents='unicode', stop_words=stopwords).fit_transform(corpus.content)
data.shape

(551, 1000)

In [31]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB

model = MultinomialNB(alpha=0.001)

precision = cross_val_score(model, data.toarray(), target, cv=10, scoring='precision').mean()
acc = cross_val_score(model, data.toarray(), target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data.toarray(), target, cv=10, scoring='recall').mean()
print(precision)
print(acc)
print(recall)

0.789107396871
0.788405095952
0.921848739496


In [32]:
model = svm.LinearSVC(C=2.15)

precision = cross_val_score(model, data.toarray(), target, cv=10, scoring='precision').mean()
acc = cross_val_score(model, data.toarray(), target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data.toarray(), target, cv=10, scoring='recall').mean()
print(precision)
print(acc)
print(recall)

0.773815562407
0.734204160619
0.835462184874


## Grid Search

In [9]:
c_range = np.logspace(-3,7,7)
param_grid = [
    {'kernel': ['rbf', 'linear'], 'C': c_range},
]
grid_search = GridSearchCV(svm.SVC(), param_grid, cv=10, verbose=3, n_jobs=10)
grid_search.fit(data, target)

Fitting 10 folds for each of 14 candidates, totalling 140 fits
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] .............. C=0.001, kernel=rbf, score=0.648148, total=   0.7s
[CV] C=0.001, kernel=linear ..........................................
[CV] .............. C=0.001, kernel=rbf, score=0.648148, total=   0.7s
[CV] .........

[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    2.6s


[CV] ........... C=0.001, kernel=linear, score=0.653846, total=   0.6s
[CV] ........... C=0.001, kernel=linear, score=0.641509, total=   0.6s
[CV] ........... C=0.001, kernel=linear, score=0.648148, total=   0.6s
[CV] C=0.0464158883361, kernel=rbf ...................................
[CV] C=0.0464158883361, kernel=rbf ...................................
[CV] C=0.0464158883361, kernel=rbf ...................................
[CV] C=0.0464158883361, kernel=rbf ...................................
[CV] .... C=0.0464158883361, kernel=rbf, score=0.648148, total=   0.9s
[CV] C=0.0464158883361, kernel=linear ................................
[CV] .... C=0.0464158883361, kernel=rbf, score=0.648148, total=   0.9s
[CV] C=0.0464158883361, kernel=linear ................................
[CV] .... C=0.0464158883361, kernel=rbf, score=0.648148, total=   0.9s
[CV] C=0.0464158883361, kernel=linear ................................
[CV] .... C=0.0464158883361, kernel=rbf, score=0.653846, total=   0.9s
[CV] C

[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   16.1s


[CV] ... C=215443.469003, kernel=linear, score=0.648148, total=   0.7s
[CV] C=10000000.0, kernel=rbf ........................................
[CV] ... C=215443.469003, kernel=linear, score=0.703704, total=   0.7s
[CV] C=10000000.0, kernel=rbf ........................................
[CV] ... C=215443.469003, kernel=linear, score=0.685185, total=   0.7s
[CV] ... C=215443.469003, kernel=linear, score=0.685185, total=   0.6s
[CV] C=10000000.0, kernel=rbf ........................................
[CV] C=10000000.0, kernel=rbf ........................................
[CV] ... C=215443.469003, kernel=linear, score=0.716981, total=   0.6s
[CV] C=10000000.0, kernel=rbf ........................................
[CV] ... C=215443.469003, kernel=linear, score=0.796296, total=   0.6s
[CV] ... C=215443.469003, kernel=linear, score=0.777778, total=   0.6s
[CV] C=10000000.0, kernel=rbf ........................................
[CV] C=10000000.0, kernel=rbf ........................................
[CV] .

[Parallel(n_jobs=10)]: Done 140 out of 140 | elapsed:   19.8s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=10,
       param_grid=[{'kernel': ['rbf', 'linear'], 'C': array([  1.00000e-03,   4.64159e-02,   2.15443e+00,   1.00000e+02,
         4.64159e+03,   2.15443e+05,   1.00000e+07])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=3)

In [10]:
print(grid_search.best_estimator_)
print(grid_search.best_score_)
print(grid_search.best_params_)

SVC(C=2.1544346900318843, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.734082397004
{'C': 2.1544346900318843, 'kernel': 'linear'}


In [11]:
model = svm.SVC(kernel='linear',C=2.15,gamma=0.1)

precision = cross_val_score(model, data, target, cv=10, scoring='precision').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(precision)
print(acc)
print(recall)

0.772455738114
0.733991829275
0.837983193277


In [29]:
nb_params = { 'alpha': np.logspace(-3, 3, 7)}

grid_search = GridSearchCV(MultinomialNB(), nb_params, cv=10, verbose=3, n_jobs=10)
grid_search.fit(data, target)

Fitting 10 folds for each of 7 candidates, totalling 70 fits
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] ...................... alpha=0.001, score=0.777778, total=   0.0s
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] ...................... alpha=0.001, score=0.722222, total=   0.0s
[CV] ...................... alpha=0.001, score=0.759259, total=   0.0s
[CV] alpha=0.001 .....................................................
[CV] alpha=0.01 

[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    0.2s


[CV] ...................... alpha=100.0, score=0.648148, total=   0.0s
[CV] ....................... alpha=10.0, score=0.648148, total=   0.0s
[CV] ...................... alpha=100.0, score=0.648148, total=   0.0s
[CV] alpha=100.0 .....................................................
[CV] alpha=10.0 ......................................................
[CV] alpha=10.0 ......................................................
[CV] ...................... alpha=100.0, score=0.648148, total=   0.0s
[CV] alpha=100.0 .....................................................
[CV] alpha=1000.0 ....................................................
[CV] ....................... alpha=10.0, score=0.648148, total=   0.0s
[CV] ...................... alpha=100.0, score=0.641509, total=   0.0s
[CV] alpha=1000.0 ....................................................
[CV] alpha=100.0 .....................................................
[CV] alpha=1000.0 ....................................................
[CV] .

[Parallel(n_jobs=10)]: Done  70 out of  70 | elapsed:    0.5s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params={}, iid=True, n_jobs=10,
       param_grid={'alpha': array([  1.00000e-03,   1.00000e-02,   1.00000e-01,   1.00000e+00,
         1.00000e+01,   1.00000e+02,   1.00000e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=3)

In [30]:
print(grid_search.best_estimator_)
print(grid_search.best_score_)
print(grid_search.best_params_)

MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)
0.788389513109
{'alpha': 0.001}


In [14]:
from sklearn.naive_bayes import BernoulliNB

nb_params = { 'alpha': np.logspace(-3, 3, 7) }

grid_search = GridSearchCV(BernoulliNB(), nb_params, cv=10, verbose=3, n_jobs=10)
grid_search.fit(data, target)

Fitting 10 folds for each of 7 candidates, totalling 70 fits
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] ...................... alpha=0.001, score=0.722222, total=   0.0s
[CV] alpha=0.001 .....................................................
[CV] ...................... alpha=0.001, score=0.796296, total=   0.0s
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] ...................... alpha=0.001, score=0.685185, total=   0.0s
[CV] alpha=0.001 .....................................................
[CV] ...................... alpha=0.001, score=0.759259, total=   0.0s
[CV] alpha=0.001 .....................................................
[CV] ...........

[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    0.2s


[CV] ...................... alpha=100.0, score=0.641509, total=   0.0s
[CV] alpha=100.0 .....................................................
[CV] alpha=1.0 .......................................................
[CV] ...................... alpha=100.0, score=0.648148, total=   0.0s
[CV] alpha=1.0 .......................................................
[CV] alpha=100.0 .....................................................
[CV] ........................ alpha=0.1, score=0.811321, total=   0.0s
[CV] alpha=100.0 .....................................................
[CV] alpha=1.0 .......................................................
[CV] ...................... alpha=100.0, score=0.653846, total=   0.0s
[CV] ...................... alpha=100.0, score=0.648148, total=   0.0s
[CV] ........................ alpha=1.0, score=0.846154, total=   0.0s
[CV] ....................... alpha=10.0, score=0.629630, total=   0.0s
[CV] alpha=100.0 .....................................................
[CV] a

[Parallel(n_jobs=10)]: Done  70 out of  70 | elapsed:    0.6s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),
       fit_params={}, iid=True, n_jobs=10,
       param_grid={'alpha': array([  1.00000e-03,   1.00000e-02,   1.00000e-01,   1.00000e+00,
         1.00000e+01,   1.00000e+02,   1.00000e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=3)

In [15]:
print(grid_search.best_estimator_)
print(grid_search.best_score_)
print(grid_search.best_params_)

BernoulliNB(alpha=0.001, binarize=0.0, class_prior=None, fit_prior=True)
0.792134831461
{'alpha': 0.001}


In [33]:
model = BernoulliNB(alpha=0.001)

precision = cross_val_score(model, data.toarray(), target, cv=10, scoring='precision').mean()
acc = cross_val_score(model, data.toarray(), target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data.toarray(), target, cv=10, scoring='recall').mean()
print(precision)
print(acc)
print(recall)

0.823478967689
0.792357415471
0.866974789916


In [24]:
from sklearn.tree import DecisionTreeClassifier 

model = DecisionTreeClassifier()

precision = cross_val_score(model, data, target, cv=10, scoring='precision').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(precision)
print(acc)
print(recall)

0.722199214086
0.643077729398
0.725798319328


In [35]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
precision = cross_val_score(model, data, target, cv=10, scoring='precision').mean()

print(precision)
print(acc)
print(recall)

0.746387998711
0.717788797506
0.803697478992


In [34]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier()

precision = cross_val_score(model, data, target, cv=10, scoring='precision').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(precision)
print(acc)
print(recall)

0.781902872052
0.707785034672
0.81218487395


In [26]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()

precision = cross_val_score(model, data, target, cv=10, scoring='precision').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(precision)
print(acc)
print(recall)

0.731940547979
0.722810836962
0.907731092437


In [27]:
from sklearn.tree import ExtraTreeClassifier

model = ExtraTreeClassifier()

precision = cross_val_score(model, data, target, cv=10, scoring='precision').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(precision)
print(acc)
print(recall)

0.706852811913
0.603202440467
0.68243697479


### confidence-weighted linear classifier (Dredze et al., 2008)

In [21]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score
import oll

## manual 10-fold cross-validation
kf = KFold(n_splits=2, random_state=None, shuffle=False)

methods = ["P" ,"AP" ,"PA" ,"PA1","PA2" ,"PAK","CW" ,"AL"]

for m in methods:
    model = oll.oll(m, C=1)

    accuracy = []
    precision = []
    recall = []
    
    for train_index, test_index in kf.split(data):

        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = target[train_index], target[test_index]
        model.fit(X_train, y_train)
        predicted = model.predict(X_test)

        accuracy.append(accuracy_score(y_test, predicted))
        precision.append(precision_score(y_test, predicted))
        recall.append(recall_score(y_test, predicted))

    print(m + ': acc(' + str(np.mean(accuracy)) 
          + '), prec(' + str(np.mean(precision))
          + '), rec(' + str(np.mean(recall)) + ')'
         )

P: acc(0.779026217228), prec(0.804893499089), rec(0.896029170709)
AP: acc(0.803370786517), prec(0.835126035397), rec(0.877061519419)
PA: acc(0.823970037453), prec(0.802823715693), rec(0.982437359917)
PA1: acc(0.823970037453), prec(0.802823715693), rec(0.982437359917)
PA2: acc(0.807116104869), prec(0.784251189424), rec(0.982437359917)
PAK: acc(0.833333333333), prec(0.82180398998), rec(0.988023952096)
CW: acc(0.850187265918), prec(0.839162112933), rec(0.973455323989)
AL: acc(0.807116104869), prec(0.822371719873), rec(0.896815307932)


In [22]:
import oll

model = oll.oll("CW", C=1)
model.fit(data, target)
predicted = model.predict(data)
scores = model.scores(data)

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(
    loss='exponential',
    n_estimators=500,
    subsample=0.5,
    max_depth=7,
    max_features=0.5,
    learning_rate=0.1
)

acc = cross_val_score(model, data.toarray(), target, cv=10, scoring='accuracy').mean()
fscore = cross_val_score(model, data.toarray(), target, cv=10, scoring='f1').mean()
print(acc)
print(fscore)

0.773041125541
0.837589051743


In [6]:
param_grid = {'learning_rate': [0.01, 0.1, 0.1, 0.5, 1.0],
              'max_depth':[1, 3, 5, 7, 9],
              'n_estimators': [100, 300, 500],
              'loss' : ['deviance', 'exponential'],
              'subsample':[0.2, 0.5, 0.8, 1],
              'max_features': [0.5, 1]}

grid_search = GridSearchCV(GradientBoostingClassifier(), param_grid=param_grid, cv=5, verbose=3, n_jobs=3, scoring='accuracy')
grid_search.fit(data.toarray(), target)

Fitting 5 folds for each of 1200 candidates, totalling 6000 fits
[CV] loss=deviance, n_estimators=100, subsample=0.2, max_depth=1, max_features=0.5, learning_rate=0.01 
[CV] loss=deviance, n_estimators=100, subsample=0.2, max_depth=1, max_features=0.5, learning_rate=0.01 
[CV] loss=deviance, n_estimators=100, subsample=0.2, max_depth=1, max_features=0.5, learning_rate=0.01 
[CV]  loss=deviance, n_estimators=100, subsample=0.2, max_depth=1, max_features=0.5, learning_rate=0.01, score=0.648649, total=   0.1s
[CV] loss=deviance, n_estimators=100, subsample=0.2, max_depth=1, max_features=0.5, learning_rate=0.01 
[CV]  loss=deviance, n_estimators=100, subsample=0.2, max_depth=1, max_features=0.5, learning_rate=0.01, score=0.648649, total=   0.1s
[CV]  loss=deviance, n_estimators=100, subsample=0.2, max_depth=1, max_features=0.5, learning_rate=0.01, score=0.654545, total=   0.1s
[CV] loss=deviance, n_estimators=100, subsample=0.5, max_depth=1, max_features=0.5, learning_rate=0.01 
[CV] loss=

[Parallel(n_jobs=3)]: Done  46 tasks      | elapsed:    5.9s


[CV]  loss=deviance, n_estimators=500, subsample=0.8, max_depth=1, max_features=0.5, learning_rate=0.01, score=0.711712, total=   0.7s
[CV] loss=deviance, n_estimators=500, subsample=0.8, max_depth=1, max_features=0.5, learning_rate=0.01 
[CV]  loss=deviance, n_estimators=500, subsample=0.5, max_depth=1, max_features=0.5, learning_rate=0.01, score=0.690909, total=   0.9s
[CV] loss=deviance, n_estimators=500, subsample=0.8, max_depth=1, max_features=0.5, learning_rate=0.01 
[CV]  loss=deviance, n_estimators=500, subsample=0.5, max_depth=1, max_features=0.5, learning_rate=0.01, score=0.733945, total=   0.9s
[CV] loss=deviance, n_estimators=500, subsample=0.8, max_depth=1, max_features=0.5, learning_rate=0.01 
[CV]  loss=deviance, n_estimators=500, subsample=0.8, max_depth=1, max_features=0.5, learning_rate=0.01, score=0.729730, total=   0.8s
[CV] loss=deviance, n_estimators=500, subsample=1, max_depth=1, max_features=0.5, learning_rate=0.01 
[CV]  loss=deviance, n_estimators=500, subsamp

[Parallel(n_jobs=3)]: Done 183 tasks      | elapsed:   39.8s


[CV] loss=deviance, n_estimators=100, subsample=0.8, max_depth=3, max_features=1, learning_rate=0.01 
[CV]  loss=deviance, n_estimators=100, subsample=0.8, max_depth=3, max_features=1, learning_rate=0.01, score=0.648649, total=   0.1s
[CV] loss=deviance, n_estimators=100, subsample=0.8, max_depth=3, max_features=1, learning_rate=0.01 
[CV]  loss=deviance, n_estimators=500, subsample=1, max_depth=3, max_features=0.5, learning_rate=0.01, score=0.816514, total=   2.4s
[CV] loss=deviance, n_estimators=100, subsample=0.8, max_depth=3, max_features=1, learning_rate=0.01 
[CV]  loss=deviance, n_estimators=100, subsample=0.8, max_depth=3, max_features=1, learning_rate=0.01, score=0.648649, total=   0.1s
[CV] loss=deviance, n_estimators=100, subsample=0.8, max_depth=3, max_features=1, learning_rate=0.01 
[CV]  loss=deviance, n_estimators=100, subsample=0.8, max_depth=3, max_features=1, learning_rate=0.01, score=0.654545, total=   0.1s
[CV] loss=deviance, n_estimators=100, subsample=1, max_depth

...

[CV]  loss=deviance, n_estimators=100, subsample=0.8, max_depth=9, max_features=1, learning_rate=1.0, score=0.706422, total=   0.1s
[CV] loss=deviance, n_estimators=100, subsample=1, max_depth=9, max_features=1, learning_rate=1.0 
[CV]  loss=deviance, n_estimators=100, subsample=1, max_depth=9, max_features=1, learning_rate=1.0, score=0.729730, total=   0.1s
[CV] loss=deviance, n_estimators=100, subsample=1, max_depth=9, max_features=1, learning_rate=1.0 
[CV]  loss=deviance, n_estimators=100, subsample=1, max_depth=9, max_features=1, learning_rate=1.0, score=0.639640, total=   0.1s
[CV] loss=deviance, n_estimators=100, subsample=1, max_depth=9, max_features=1, learning_rate=1.0 
[CV]  loss=deviance, n_estimators=100, subsample=1, max_depth=9, max_features=1, learning_rate=1.0, score=0.672727, total=   0.1s
[CV] loss=deviance, n_estimators=300, subsample=0.2, max_depth=9, max_features=1, learning_rate=1.0 
[CV]  loss=deviance, n_estimators=100, subsample=1, max_depth=9, max_features=1,

[Parallel(n_jobs=3)]: Done 5916 tasks      | elapsed: 33.6min


[CV]  loss=exponential, n_estimators=300, subsample=1, max_depth=9, max_features=0.5, learning_rate=1.0, score=0.738739, total=   0.6s
[CV] loss=exponential, n_estimators=300, subsample=1, max_depth=9, max_features=0.5, learning_rate=1.0 
[CV]  loss=exponential, n_estimators=300, subsample=1, max_depth=9, max_features=0.5, learning_rate=1.0, score=0.700000, total=   0.6s
[CV] loss=exponential, n_estimators=500, subsample=0.2, max_depth=9, max_features=0.5, learning_rate=1.0 
[CV]  loss=exponential, n_estimators=300, subsample=1, max_depth=9, max_features=0.5, learning_rate=1.0, score=0.727273, total=   0.6s
[CV] loss=exponential, n_estimators=500, subsample=0.2, max_depth=9, max_features=0.5, learning_rate=1.0 
[CV]  loss=exponential, n_estimators=300, subsample=1, max_depth=9, max_features=0.5, learning_rate=1.0, score=0.761468, total=   0.6s
[CV] loss=exponential, n_estimators=500, subsample=0.2, max_depth=9, max_features=0.5, learning_rate=1.0 
[CV]  loss=exponential, n_estimators=5

[Parallel(n_jobs=3)]: Done 6000 out of 6000 | elapsed: 33.8min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'loss': ['deviance', 'exponential'], 'n_estimators': [100, 300, 500], 'subsample': [0.2, 0.5, 0.8, 1], 'max_depth': [1, 3, 5, 7, 9], 'max_features': [0.5, 1], 'learning_rate': [0.01, 0.1, 0.1, 0.5, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=3)

In [7]:
print(grid_search.best_estimator_)
print(grid_search.best_score_)
print(grid_search.best_params_)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='exponential', max_depth=7,
              max_features=0.5, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=500, presort='auto', random_state=None,
              subsample=0.5, verbose=0, warm_start=False)
0.794918330309
{'loss': 'exponential', 'n_estimators': 500, 'subsample': 0.5, 'max_depth': 7, 'max_features': 0.5, 'learning_rate': 0.1}
